# 修了課題DEMO②　MNIST

##はじめに
MNISTとは手書き数字の画像データセットのことです。

今回はそのMNISTから画像を学習して数字を精度が高く識別できるモデルを作成してみましょう。

今回はDEMOとして、手書き認識モデルとして一般的なDeepConvNetを取り上げました。

DeepConvNetのネットワーク構成は
    
    conv - relu - conv- relu - pool -
    conv - relu - conv- relu - pool -
    conv - relu - conv- relu - pool -
    affine - relu - dropout - affine - dropout - SoftmaxWithLoss

上記のように計２１層となっています。

##作成までの流れ
大まかな流れとして
1. データのダウンロードと成形

   データダウンロードしてそのまま活用するのは困難です。
   モデルが学習できるよう、カテゴリ変数に置き換えたり、
   欠損値を補完したりなど、成形する必要があります。
2. モデルの構築

   学習を行うモデルのアルゴリズムを理解して、コードを作成します。

3. 学習と結果

   学習を行い結果を確認してみます。
   精度が目標まで達したら、提出用のデータを作成します。

## 必要なライブラリーのインポート

In [ ]:
# GPUを利用する場合
import cupy as np
# CPUを利用する場合
# import numpy as np

from sklearn.datasets import fetch_openml
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split

import matplotlib.pyplot as plt
from collections import OrderedDict
import pickle

#1.データのダウンロードと成形

## データのダウンロード

今回はDeepConvNetの入力形式に合わせるため、

正規化を行った後にそれぞれの画像の次元に(28,28)の次元を追加しています。

In [ ]:
# MNISTデータのロードと訓練データ、テストデータの分割を行う
def load_mnist():
    mnist = fetch_openml('MNIST_784', version=1, data_home="MNIST_data/")
    mnist_X, mnist_y = shuffle(mnist.data[:60000].astype('float32'),
                               mnist.target[:60000].astype('int32'), random_state=42)

    # 正規化を行う
    mnist_X = mnist_X / 255.0
    mnist_X = np.array(mnist_X)
    train_images = []
    #次元を追加する
    for i in range(mnist_X.shape[0]):
        train_images.append(np.expand_dims(mnist_X[i].reshape(28, 28), axis=0))
    mnist_X = np.array(train_images)

    # one-hot表現に変換する
    mnist_y = np.eye(10)[np.array(mnist_y)]

    return train_test_split(mnist_X, mnist_y,
                test_size=0.2,
                random_state=42)

In [ ]:
# それぞれのデータの形式を出力する
x_train, x_val, y_train, y_val = load_mnist()
print('学習データ ', x_train.shape)
print('学習データの正解ラベル ', y_train.shape)
print('検証データ ', x_val.shape)
print('検証データの正解ラベル ', y_val.shape)

# 2.モデルの構築

## 必要な基本関数の定義

In [ ]:
# シグモイド関数
def sigmoid(x):
    return 1 / (1 + np.exp(-x))

# シグモイド関数の微分
def sigmoid_grad(x):
    return (1.0 - sigmoid(x)) * sigmoid(x)

# relu関数
def relu(x):
    return np.maximum(0, x)

# relu関数の微分
def relu_grad(x):
    grad = np.zeros_like(x)
    grad[x>=0] = 1
    return grad

# ソフトマックス関数
def softmax(x):
    x = x - np.max(x, axis=-1, keepdims=True)   # オーバーフロー対策
    return np.exp(x) / np.sum(np.exp(x), axis=-1, keepdims=True)

# 交差エントロピー誤差
def cross_entropy_error(y, t):
    if y.ndim == 1:
        t = t.reshape(1, t.size)
        y = y.reshape(1, y.size)

    # 教師データがone-hot-vectorの場合、正解ラベルのインデックスに変換
    if t.size == y.size:
        t = t.argmax(axis=1)

    batch_size = y.shape[0]
    return -np.sum(np.log(y[np.arange(batch_size), t] + 1e-7)) / batch_size

# ソフトマックス損失
def softmax_loss(X, t):
    y = softmax(X)
    return cross_entropy_error(y, t)

In [ ]:
# Relu関数のクラス
class Relu:
    def __init__(self):
        self.mask = None

    def forward(self, x):
        self.mask = (x <= 0)
        out = x.copy()
        out[self.mask] = 0

        return out

    def backward(self, dout):
        dout[self.mask] = 0
        dx = dout

        return dx

# シグモイド関数のクラス
class Sigmoid:
    def __init__(self):
        self.out = None

    def forward(self, x):
        out = sigmoid(x)
        self.out = out
        return out

    def backward(self, dout):
        dx = dout * (1.0 - self.out) * self.out

        return dx

# アフィン層(全結合層)のクラス
class Affine:
    def __init__(self, W, b):
        self.W =W
        self.b = b

        self.x = None
        self.original_x_shape = None
        # 重み・バイアスパラメータの微分
        self.dW = None
        self.db = None

    def forward(self, x):
        # テンソル対応
        self.original_x_shape = x.shape
        x = x.reshape(x.shape[0], -1)
        self.x = x

        out = np.dot(self.x, self.W) + self.b

        return out

    def backward(self, dout):
        dx = np.dot(dout, self.W.T)
        self.dW = np.dot(self.x.T, dout)
        self.db = np.sum(dout, axis=0)

        dx = dx.reshape(*self.original_x_shape)  # 入力データの形状に戻す（テンソル対応）
        return dx

# ソフトマックス損失層のクラス
class SoftmaxWithLoss:
    def __init__(self):
        self.loss = None
        self.y = None # softmaxの出力
        self.t = None # 教師データ

    def forward(self, x, t):
        self.t = t
        self.y = softmax(x)
        self.loss = cross_entropy_error(self.y, self.t)

        return self.loss

    def backward(self, dout=1):
        batch_size = self.t.shape[0]
        if self.t.size == self.y.size: # 教師データがone-hot-vectorの場合
            dx = (self.y - self.t) / batch_size
        else:
            dx = self.y.copy()
            dx[np.arange(batch_size), self.t] -= 1
            dx = dx / batch_size

        return dx

# 確率的勾配降下法（Stochastic Gradient Descent)のクラス
class SGD:
    def __init__(self, lr=0.01):
        # 学習率を初期化する（0.01にする）。
        self.lr = lr

    def update(self, params, grads):
        # パラメータの更新を行うメソッド
        # params: 更新されるパラメータ（例：ニューラルネットワークの重みやバイアス）
        # grads: 各パラメータに関する勾配（損失関数の勾配）
        for key in params.keys():
            # 各パラメータに対して、勾配に基づいて更新を行う。
            params[key] -= self.lr * grads[key]

# ドロップアウト層の関数
class Dropout:
    def __init__(self, dropout_ratio=0.5):
        # ドロップアウトの割合を設定する（デフォルトは0.5）
        self.dropout_ratio = dropout_ratio
        self.mask = None

    def forward(self, x, train_flg=True):
        if train_flg:
            # 訓練時は、ランダムにノードをドロップアウトする。
            self.mask = np.random.rand(*x.shape) > self.dropout_ratio
            # マスクを適用して出力を返す。
            return x * self.mask
        else:
            # 評価時は、ドロップアウトは適用せず、
            # 各ノードの出力をドロップアウト比率でスケーリングする。
            return x * (1.0 - self.dropout_ratio)

    def backward(self, dout):
        # 逆伝播時にもマスクを適用する。
        # これにより、前方伝播時にドロップアウトされたノードは、
        # 逆伝播でも影響を与えない。
        return dout * self.mask

# バッチノーマライゼーションのクラス
class BatchNormalization:
    def __init__(self, gamma, beta, momentum=0.9, running_mean=None, running_var=None):
        self.gamma = gamma
        self.beta = beta
        self.momentum = momentum
        self.input_shape = None # Conv層の場合は4次元、全結合層の場合は2次元

        # テスト時に使用する平均と分散
        self.running_mean = running_mean
        self.running_var = running_var

        # backward時に使用する中間データ
        self.batch_size = None
        self.xc = None
        self.std = None
        self.dgamma = None
        self.dbeta = None

    def forward(self, x, train_flg=True):
        self.input_shape = x.shape
        if x.ndim != 2:
            N, C, H, W = x.shape
            x = x.reshape(N, -1)

        out = self.__forward(x, train_flg)

        return out.reshape(*self.input_shape)

    def __forward(self, x, train_flg):
        if self.running_mean is None:
            N, D = x.shape
            self.running_mean = np.zeros(D)
            self.running_var = np.zeros(D)

        if train_flg:
            mu = x.mean(axis=0)
            xc = x - mu
            var = np.mean(xc**2, axis=0)
            std = np.sqrt(var + 10e-7)
            xn = xc / std

            self.batch_size = x.shape[0]
            self.xc = xc
            self.xn = xn
            self.std = std
            self.running_mean = self.momentum * self.running_mean + (1-self.momentum) * mu
            self.running_var = self.momentum * self.running_var + (1-self.momentum) * var
        else:
            xc = x - self.running_mean
            xn = xc / ((np.sqrt(self.running_var + 10e-7)))

        out = self.gamma * xn + self.beta
        return out

    def backward(self, dout):
        if dout.ndim != 2:
            N, C, H, W = dout.shape
            dout = dout.reshape(N, -1)

        dx = self.__backward(dout)

        dx = dx.reshape(*self.input_shape)
        return dx

    def __backward(self, dout):
        dbeta = dout.sum(axis=0)
        dgamma = np.sum(self.xn * dout, axis=0)
        dxn = self.gamma * dout
        dxc = dxn / self.std
        dstd = -np.sum((dxn * self.xc) / (self.std * self.std), axis=0)
        dvar = 0.5 * dstd / self.std
        dxc += (2.0 / self.batch_size) * self.xc * dvar
        dmu = np.sum(dxc, axis=0)
        dx = dxc - dmu / self.batch_size

        self.dgamma = dgamma
        self.dbeta = dbeta

        return dx

## im2col関数とcol2im関数の実装

In [ ]:
# im2col関数
def im2col(input_data, filter_h, filter_w, stride=1, pad=0):
    N, C, H, W = input_data.shape
    out_h = (H + 2*pad - filter_h)//stride + 1
    out_w = (W + 2*pad - filter_w)//stride + 1

    img = np.pad(input_data, [(0,0), (0,0), (pad, pad), (pad, pad)], 'constant')
    col = np.zeros((N, C, filter_h, filter_w, out_h, out_w))

    for y in range(filter_h):
        y_max = y + stride*out_h
        for x in range(filter_w):
            x_max = x + stride*out_w
            col[:, :, y, x, :, :] = img[:, :, y:y_max:stride, x:x_max:stride]

    col = col.transpose(0, 4, 5, 1, 2, 3).reshape(N*out_h*out_w, -1)
    return col

# col2im関数
def col2im(col, input_shape, filter_h, filter_w, stride=1, pad=0):
    N, C, H, W = input_shape
    out_h = (H + 2*pad - filter_h)//stride + 1
    out_w = (W + 2*pad - filter_w)//stride + 1
    col = col.reshape(N, out_h, out_w, C, filter_h, filter_w).transpose(0, 3, 4, 5, 1, 2)

    img = np.zeros((N, C, H + 2*pad + stride - 1, W + 2*pad + stride - 1))
    for y in range(filter_h):
        y_max = y + stride*out_h
        for x in range(filter_w):
            x_max = x + stride*out_w
            img[:, :, y:y_max:stride, x:x_max:stride] += col[:, :, y, x, :, :]

    return img[:, :, pad:H + pad, pad:W + pad]

## Convlution層とプーリング層の実装

In [ ]:
# Convlution層のクラス
class Convolution:
    def __init__(self, W, b, stride=1, pad=0):
        self.W = W
        self.b = b
        self.stride = stride
        self.pad = pad

        # 中間データ（backward時に使用）
        self.x = None
        self.col = None
        self.col_W = None

        # 重み・バイアスパラメータの勾配
        self.dW = None
        self.db = None

    def forward(self, x):
        FN, C, FH, FW = self.W.shape
        N, C, H, W = x.shape
        out_h = 1 + int((H + 2*self.pad - FH) / self.stride)
        out_w = 1 + int((W + 2*self.pad - FW) / self.stride)

        col = im2col(x, FH, FW, self.stride, self.pad)
        col_W = self.W.reshape(FN, -1).T

        out = np.dot(col, col_W) + self.b
        out = out.reshape(N, out_h, out_w, -1).transpose(0, 3, 1, 2)

        self.x = x
        self.col = col
        self.col_W = col_W

        return out

    def backward(self, dout):
        FN, C, FH, FW = self.W.shape
        dout = dout.transpose(0,2,3,1).reshape(-1, FN)

        self.db = np.sum(dout, axis=0)
        self.dW = np.dot(self.col.T, dout)
        self.dW = self.dW.transpose(1, 0).reshape(FN, C, FH, FW)

        dcol = np.dot(dout, self.col_W.T)
        dx = col2im(dcol, self.x.shape, FH, FW, self.stride, self.pad)

        return dx

# プーリング層のクラス
class Pooling:
    def __init__(self, pool_h, pool_w, stride=2, pad=0):
        self.pool_h = pool_h
        self.pool_w = pool_w
        self.stride = stride
        self.pad = pad

        self.x = None
        self.arg_max = None

    def forward(self, x):
        N, C, H, W = x.shape
        out_h = int(1 + (H - self.pool_h) / self.stride)
        out_w = int(1 + (W - self.pool_w) / self.stride)

        col = im2col(x, self.pool_h, self.pool_w, self.stride, self.pad)
        col = col.reshape(-1, self.pool_h*self.pool_w)

        arg_max = np.argmax(col, axis=1)
        out = np.max(col, axis=1)
        out = out.reshape(N, out_h, out_w, C).transpose(0, 3, 1, 2)

        self.x = x
        self.arg_max = arg_max

        return out

    def backward(self, dout):
        dout = dout.transpose(0, 2, 3, 1)

        pool_size = self.pool_h * self.pool_w
        dmax = np.zeros((dout.size, pool_size))
        dmax[np.arange(self.arg_max.size), self.arg_max.flatten()] = dout.flatten()
        dmax = dmax.reshape(dout.shape + (pool_size,))

        dcol = dmax.reshape(dmax.shape[0] * dmax.shape[1] * dmax.shape[2], -1)
        dx = col2im(dcol, self.x.shape, self.pool_h, self.pool_w, self.stride, self.pad)

        return dx

## CNNの実装

In [ ]:
class DeepConvNet:
    def __init__(self, input_dim=(1, 28, 28),
                 conv_param_1 = {'filter_num':16, 'filter_size':3, 'pad':1, 'stride':1},
                 conv_param_2 = {'filter_num':16, 'filter_size':3, 'pad':1, 'stride':1},
                 conv_param_3 = {'filter_num':32, 'filter_size':3, 'pad':1, 'stride':1},
                 conv_param_4 = {'filter_num':32, 'filter_size':3, 'pad':2, 'stride':1},
                 conv_param_5 = {'filter_num':64, 'filter_size':3, 'pad':1, 'stride':1},
                 conv_param_6 = {'filter_num':64, 'filter_size':3, 'pad':1, 'stride':1},
                 hidden_size=50, output_size=10):
        # 重みの初期化===========
        # 各層のニューロンひとつあたりが、前層のニューロンといくつのつながりがあるか
        pre_node_nums = np.array([1*3*3, 16*3*3, 16*3*3, 32*3*3, 32*3*3, 64*3*3, 64*4*4, hidden_size])
        weight_init_scales = np.sqrt(2.0 / pre_node_nums)  # ReLUを使う場合に推奨される初期値

        self.params = {}
        pre_channel_num = input_dim[0]
        for idx, conv_param in enumerate([conv_param_1, conv_param_2, conv_param_3, conv_param_4, conv_param_5, conv_param_6]):
            self.params['W' + str(idx+1)] = weight_init_scales[idx] * np.random.randn(conv_param['filter_num'], pre_channel_num, conv_param['filter_size'], conv_param['filter_size'])
            self.params['b' + str(idx+1)] = np.zeros(conv_param['filter_num'])
            pre_channel_num = conv_param['filter_num']
        self.params['W7'] = weight_init_scales[6] * np.random.randn(64*4*4, hidden_size)
        self.params['b7'] = np.zeros(hidden_size)
        self.params['W8'] = weight_init_scales[7] * np.random.randn(hidden_size, output_size)
        self.params['b8'] = np.zeros(output_size)

        # レイヤの生成===========
        self.layers = []
        self.layers.append(Convolution(self.params['W1'], self.params['b1'],
                           conv_param_1['stride'], conv_param_1['pad']))
        self.layers.append(Relu())
        self.layers.append(Convolution(self.params['W2'], self.params['b2'],
                           conv_param_2['stride'], conv_param_2['pad']))
        self.layers.append(Relu())
        self.layers.append(Pooling(pool_h=2, pool_w=2, stride=2))
        self.layers.append(Convolution(self.params['W3'], self.params['b3'],
                           conv_param_3['stride'], conv_param_3['pad']))
        self.layers.append(Relu())
        self.layers.append(Convolution(self.params['W4'], self.params['b4'],
                           conv_param_4['stride'], conv_param_4['pad']))
        self.layers.append(Relu())
        self.layers.append(Pooling(pool_h=2, pool_w=2, stride=2))
        self.layers.append(Convolution(self.params['W5'], self.params['b5'],
                           conv_param_5['stride'], conv_param_5['pad']))
        self.layers.append(Relu())
        self.layers.append(Convolution(self.params['W6'], self.params['b6'],
                           conv_param_6['stride'], conv_param_6['pad']))
        self.layers.append(Relu())
        self.layers.append(Pooling(pool_h=2, pool_w=2, stride=2))
        self.layers.append(Affine(self.params['W7'], self.params['b7']))
        self.layers.append(Relu())
        self.layers.append(Dropout(0.5))
        self.layers.append(Affine(self.params['W8'], self.params['b8']))
        self.layers.append(Dropout(0.5))

        self.last_layer = SoftmaxWithLoss()

    def predict(self, x, train_flg=False):
        # ネットワークを通して入力データxを順伝播させる。
        # train_flgがFalseの場合、モデルは評価モードで動作する。

        for layer in self.layers:
            # ネットワークの各層を順に処理する。
            if isinstance(layer, Dropout):
                # 層がドロップアウト層の場合、
                # train_flgに応じてドロップアウトを適用するか決定する。
                x = layer.forward(x, train_flg)
            else:
                # ドロップアウト層でない場合、通常の順伝播を行う。
                x = layer.forward(x)
        return x

    def loss(self, x, t):
        # ネットワークを通じて入力データxを順伝播させ、予測値yを得る。
        y = self.predict(x, train_flg=True)

        # 最終層（通常は損失層）を使用して、
        # 予測値yと真のターゲットtとの間の損失を計算する。
        return self.last_layer.forward(y, t)

    def accuracy(self, x, t, batch_size=100):
        # ターゲットデータtがone-hotエンコーディングされている場合、
        # 最大値のインデックス（クラスラベル）に変換する。
        if t.ndim != 1 : t = np.argmax(t, axis=1)

        acc = 0.0

         # バッチ処理を行い、各バッチに対する精度を計算
        for i in range(int(x.shape[0] / batch_size)):
            # バッチを抽出
            tx = x[i*batch_size:(i+1)*batch_size]
            tt = t[i*batch_size:(i+1)*batch_size]

            # 予測を計算（評価モードなので、train_flgはFalse）
            y = self.predict(tx, train_flg=False)
            # 予測結果の最大値のインデックスを取得
            y = np.argmax(y, axis=1)

            # 正解数を加算
            acc += np.sum(y == tt)

        # 全データに対する精度を計算して返す
        return acc / x.shape[0]

    def gradient(self, x, t):
        # forward
        self.loss(x, t)

        # backward
        dout = 1
        dout = self.last_layer.backward(dout)

        tmp_layers = self.layers.copy()
        tmp_layers.reverse()
        for layer in tmp_layers:
            dout = layer.backward(dout)

        # 設定
        grads = {}
        for i, layer_idx in enumerate((0, 2, 5, 7, 10, 12, 15, 18)):
            grads['W' + str(i+1)] = self.layers[layer_idx].dW
            grads['b' + str(i+1)] = self.layers[layer_idx].db

        return grads

    def save_params(self, file_name="params.pkl"):
        params = {}
        for key, val in self.params.items():
            params[key] = val
        with open(file_name, 'wb') as f:
            pickle.dump(params, f)

    def load_params(self, file_name="params.pkl"):
        with open(file_name, 'rb') as f:
            params = pickle.load(f)
        for key, val in params.items():
            self.params[key] = val

        for i, layer_idx in enumerate((0, 2, 5, 7, 10, 12, 15, 18)):
            self.layers[layer_idx].W = self.params['W' + str(i+1)]
            self.layers[layer_idx].b = self.params['b' + str(i+1)]

## 学習用のクラス

In [ ]:
# ニューラルネットの訓練を行うクラス
class Trainer:

    def __init__(self, network, x_train, t_train, x_val, t_val,
                 epochs=20, mini_batch_size=100,
                 optimizer='SGD', optimizer_param={'lr':0.01},
                 evaluate_sample_num_per_epoch=None, verbose=True):
        self.network = network
        self.verbose = verbose
        self.x_train = x_train
        self.t_train = t_train
        self.x_val = x_val
        self.t_val = t_val
        self.epochs = epochs
        self.batch_size = mini_batch_size
        self.evaluate_sample_num_per_epoch = evaluate_sample_num_per_epoch

        # optimizer
        optimizer_class_dict = {'sgd':SGD}
        self.optimizer = optimizer_class_dict[optimizer.lower()](**optimizer_param)

        self.train_size = x_train.shape[0]
        self.iter_per_epoch = max(self.train_size / mini_batch_size, 1)
        self.max_iter = int(epochs * self.iter_per_epoch)
        self.current_iter = 0
        self.current_epoch = 0

        self.train_loss_list = []
        self.train_acc_list = []
        self.val_acc_list = []

    def train_step(self):
        batch_mask = np.random.choice(self.train_size, self.batch_size)
        x_batch = self.x_train[batch_mask]
        t_batch = self.t_train[batch_mask]

        grads = self.network.gradient(x_batch, t_batch)
        self.optimizer.update(self.network.params, grads)

        loss = self.network.loss(x_batch, t_batch)
        self.train_loss_list.append(loss)
        if self.verbose: print("train loss:" + str(loss))

        if self.current_iter % self.iter_per_epoch == 0:
            self.current_epoch += 1

            x_train_sample, t_train_sample = self.x_train, self.t_train
            x_val_sample, t_val_sample = self.x_val, self.t_val
            if not self.evaluate_sample_num_per_epoch is None:
                t = self.evaluate_sample_num_per_epoch
                x_train_sample, t_train_sample = self.x_train[:t], self.t_train[:t]
                x_val_sample, t_val_sample = self.x_val[:t], self.t_val[:t]

            train_acc = self.network.accuracy(x_train_sample, t_train_sample)
            val_acc = self.network.accuracy(x_val_sample, t_val_sample)
            self.train_acc_list.append(train_acc)
            self.val_acc_list.append(val_acc)

            if self.verbose: print("=== epoch:" + str(self.current_epoch) + ", train acc:" + str(train_acc) + ", val acc:" + str(val_acc) + " ===")
        self.current_iter += 1

    def train(self):
        for i in range(self.max_iter):
            self.train_step()
            if i % self.iter_per_epoch == 0:
              print(int(i//self.iter_per_epoch)+1, 'epoch')
              val_acc = self.network.accuracy(self.x_val, self.t_val)
              print("val acc:" + str(val_acc))

        val_acc = self.network.accuracy(self.x_val, self.t_val)
        print("=============== 最終 val Accuracy ===============")
        print("val acc:" + str(val_acc))

#3.学習から提出まで

## 学習

In [ ]:
# DeepConvNetネットワークモデルのインスタンスを作成する。
network = DeepConvNet()

# Trainerクラスのインスタンスを作成し、ネットワークの訓練を行う。
trainer = Trainer(network, x_train, y_train, x_val, y_val,
                  epochs=10,            # 学習を行うエポック数
                  mini_batch_size=200,  # ミニバッチのサイズ
                  optimizer='SGD',      # 最適化アルゴリズムとして確率的勾配降下法（SGD）を使用
                  optimizer_param={'lr':0.001},   # SGDの学習率
                  evaluate_sample_num_per_epoch=1000,   # 各エポックで評価するサンプル数
                  verbose=False)        # 詳細な出力を表示しない

# 学習を開始する。
trainer.train()

今回のモデルの仕様では目標の精度には達していませんね。

それぞれの層がどのような働きをしているか、インポートするライブラリを増やして、より高性能のモデルを考えてみると

精度が向上するヒントが見つかるかもしれません。

## （参考）提出用データの作成の仕方

上記のDeepConvNetでテスト用データを予測して提出用ファイルを出力するまでを
掲載してみました。

このcsvファイルを「修了課題提出用サイト」にアップロードすると結果を確認することができます。

In [ ]:
import locale
def getpreferredencoding(do_setlocale = True):
    return "UTF-8"
locale.getpreferredencoding = getpreferredencoding

# テスト用のデータ（正解ラベルなし）のダウンロード
!wget 'https://drive.google.com/uc?export=download&id=18h1BsXvC6q_yZsO_TcONyotw5wXEU9b8' -O mnist_x_test.csv

In [ ]:
import pandas as pd
import cupy as np
x_test = pd.read_csv('mnist_x_test.csv', index_col=0)
x_test = x_test.values
x_test.shape

In [ ]:
# データの前処理

# 画像データを正規化する（ピクセル値を0から1の範囲に変換）
# MNISTの画像のピクセル値は0から255の範囲のため、255で割ります。
x_test = x_test / 255.0

# リスト形式のデータをNumPy配列に変換する。
x_test = np.array(x_test)

# 画像データを格納するための空のリストを作成する。
train_images = []

# 各画像データに対して処理を行う。
for i in range(x_test.shape[0]):
    # 画像データを28x28の2次元配列に変換し、np.expand_dimsを使用して次元を追加する。
    # これにより、画像は1x28x28の3次元配列にする。
    train_images.append(np.expand_dims(x_test[i].reshape(28, 28), axis=0))

# リスト形式の画像データをNumPy配列に変換する。
x_test = np.array(train_images)

# 最終的なデータの形状を出力します。
x_test.shape

In [ ]:
# データサイズが大きく、メモリがオーバーするためバッチで予測する

# ミニバッチのサイズを設定する。
mini_batch = 100

# データをミニバッチに分割して処理する。
for mini_x_test in range(1, len(x_test)//mini_batch+1):

  # ミニバッチの開始と終了インデックスを計算する。
  start = (mini_x_test-1)*mini_batch
  end = mini_x_test*mini_batch

  # ネットワークを使用して予測を行う。
  y_pred = network.predict(x_test[start:end])

  # 最初のミニバッチの場合、予測結果をy_predsに格納する。
  # そうでない場合、以前の予測結果と新しい予測結果を結合する。
  if mini_x_test == 1:
    y_preds = y_pred
  else:
    y_preds = np.concatenate((y_preds, y_pred), axis=0)

# 各予測の最大値のインデックス（クラスラベル）を取得する。
y_preds = np.argmax(y_preds, axis=1)

# 最終的な予測結果の形状を出力する。
y_preds.shape

In [ ]:
# csvファイルで出力する
pd.DataFrame(y_preds.get(), columns=['number']).to_csv('y_pred.csv')